In [ ]:
! pip install transformers datasets faiss-cpu scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 66.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-

In [ ]:
!pip install rank_bm25

In [ ]:
from datasets import load_dataset
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
import torch
nltk.download('punkt')

2025-04-21 19:08:13.915088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745262494.171405      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745262494.250044      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")["train"]
dataset

README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

Dataset({
    features: ['qtype', 'Question', 'Answer'],
    num_rows: 16407
})

In [ ]:
questions = dataset["Question"]
answers = dataset["Answer"]

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
answer_embeddings = model.encode(answers, convert_to_tensor=True)

Batches:   0%|          | 0/513 [00:00<?, ?it/s]

In [ ]:
query = "Who is at risk for Lymphocytic Choriomeningitis (LCM) ?"
query_embedding = model.encode(query, convert_to_tensor=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
cosine_scores = util.pytorch_cos_sim(query_embedding, answer_embeddings)[0]

In [ ]:
top_k = 5
top_results = torch.topk(cosine_scores, k=top_k)

print(f"\n🔍 Query: {query}\n")
print("📚 Top 5 Relevant Answers:\n")
for score, idx in zip(top_results[0], top_results[1]):
    print(f"[Score: {score:.4f}]")
    print(f"Answer: {answers[idx]}")
    print(f"Matched Question: {questions[idx]}")
    print("-" * 90)


🔍 Query: Who is at risk for Lymphocytic Choriomeningitis (LCM) ?

📚 Top 5 Relevant Answers:

[Score: 0.5409]
Answer: LCH is a rare condition, although its prevalence is unknown.
Matched Question: How many people are affected by lissencephaly with cerebellar hypoplasia ?
------------------------------------------------------------------------------------------
[Score: 0.4964]
Answer: It is estimated that 500 to 1,000 people worldwide have chorea-acanthocytosis.
Matched Question: How many people are affected by chorea-acanthocytosis ?
------------------------------------------------------------------------------------------
[Score: 0.4948]
Answer: Langerhans cell histiocytosis (LCH) is a disorder that primarily affects children, but is also found in adults of all ages. People with LCH produce too many Langerhans cells or histiocytes, a form of white blood cell found in healthy people that is supposed to protect the body from infection. In people with LCH, these cells multiply excessivel